### Data import

In [1]:
import pandas as pd

processed_data = pd.read_csv('data/occurences_processed_data.csv')
processed_data.head(3)

,Unnamed: 0,latitude,longitude,country,region,img_path
0,0,20.824885,-98.499517,Mexico,Hidalgo,0.png
1,1,-3.451752,-54.563937,Unknown,Unknown,1.png
2,2,-23.496464,-47.460542,Brazil,São Paulo,2.png


In [3]:
processed_data['country'].nunique()

45

In [2]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50

# Charger un modèle CNN pré-entraîné
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(45, activation='softmax')  # num_classes = nombre de pays
])




Matplotlib is building the font cache; this may take a moment.
